In [1]:
# import required libraries 

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [2]:
# Load the dataset
data = pd.read_csv("parkinsons.data")  # Replace with your actual dataset filename

# Separate features and target
X = data.drop(['name', 'status'], axis=1)  # Remove 'name' and 'status' columns
y = data['status']

In [3]:
# Train-Test Split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# First, split the data into train_val and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Next, split the train_val set into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

# Now you have X_train, X_val, X_test, y_train, y_val, and y_test

In [4]:
# Analysis of train-test split

train_status_counts = y_train.value_counts()
test_status_counts = y_test.value_counts()

print("Train set status distribution:")
print(train_status_counts)

print("\nTest set status distribution:")
print(test_status_counts)

# status explanation:
# 0 = healthy 
# 1 = non-healthy (PD - Parkinson's disease)

Train set status distribution:
1    88
0    29
Name: status, dtype: int64

Test set status distribution:
1    32
0     7
Name: status, dtype: int64


In [5]:
# Scale the numerical features

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)

In [6]:
# Create an XGBoost classifier
model = xgb.XGBClassifier(random_state=42)
# Train the model on the training data
#model.fit(X_train_scaled, y_train)

# Create an XGBoost classifier with early stopping in the constructor
# model = xgb.XGBClassifier(random_state=42, early_stopping_rounds=10)

# Train the model on the training data with validation set
#model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=True)
model = xgb.XGBClassifier(random_state=42)

# Define hyperparameter distribution for random search
param_dist = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': range(100, 1000, 100),
    'max_depth': [3, 4, 5, 6],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2],
}

# Perform random search using cross-validation
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=5, scoring='accuracy', cv=3, random_state=42, verbose=2, error_score='raise')
random_search.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)])

# Print the best hyperparameters
print("Best Hyperparameters:", random_search.best_params_)


Fitting 3 folds for each of 5 candidates, totalling 15 fits
[0]	validation_0-logloss:0.64771
[1]	validation_0-logloss:0.61100
[2]	validation_0-logloss:0.57986
[3]	validation_0-logloss:0.55441
[4]	validation_0-logloss:0.53284
[5]	validation_0-logloss:0.51073
[6]	validation_0-logloss:0.49538
[7]	validation_0-logloss:0.47824
[8]	validation_0-logloss:0.46420
[9]	validation_0-logloss:0.45402
[10]	validation_0-logloss:0.44116
[11]	validation_0-logloss:0.43434
[12]	validation_0-logloss:0.42846
[13]	validation_0-logloss:0.42396
[14]	validation_0-logloss:0.41875
[15]	validation_0-logloss:0.41148
[16]	validation_0-logloss:0.41038
[17]	validation_0-logloss:0.40850
[18]	validation_0-logloss:0.40624
[19]	validation_0-logloss:0.40513
[20]	validation_0-logloss:0.40318
[21]	validation_0-logloss:0.40236
[22]	validation_0-logloss:0.40218
[23]	validation_0-logloss:0.40237
[24]	validation_0-logloss:0.40264
[25]	validation_0-logloss:0.40358
[26]	validation_0-logloss:0.40274
[27]	validation_0-logloss:0.4008

[236]	validation_0-logloss:0.33264
[237]	validation_0-logloss:0.33264
[238]	validation_0-logloss:0.33264
[239]	validation_0-logloss:0.33264
[240]	validation_0-logloss:0.33264
[241]	validation_0-logloss:0.33264
[242]	validation_0-logloss:0.33264
[243]	validation_0-logloss:0.33264
[244]	validation_0-logloss:0.33264
[245]	validation_0-logloss:0.33264
[246]	validation_0-logloss:0.33264
[247]	validation_0-logloss:0.33264
[248]	validation_0-logloss:0.33264
[249]	validation_0-logloss:0.33264
[250]	validation_0-logloss:0.33264
[251]	validation_0-logloss:0.33264
[252]	validation_0-logloss:0.33264
[253]	validation_0-logloss:0.33264
[254]	validation_0-logloss:0.33264
[255]	validation_0-logloss:0.33264
[256]	validation_0-logloss:0.33264
[257]	validation_0-logloss:0.33264
[258]	validation_0-logloss:0.33264
[259]	validation_0-logloss:0.33264
[260]	validation_0-logloss:0.33264
[261]	validation_0-logloss:0.33264
[262]	validation_0-logloss:0.33264
[263]	validation_0-logloss:0.33264
[264]	validation_0-l

[471]	validation_0-logloss:0.33264
[472]	validation_0-logloss:0.33264
[473]	validation_0-logloss:0.33264
[474]	validation_0-logloss:0.33264
[475]	validation_0-logloss:0.33264
[476]	validation_0-logloss:0.33264
[477]	validation_0-logloss:0.33264
[478]	validation_0-logloss:0.33264
[479]	validation_0-logloss:0.33264
[480]	validation_0-logloss:0.33264
[481]	validation_0-logloss:0.33264
[482]	validation_0-logloss:0.33264
[483]	validation_0-logloss:0.33264
[484]	validation_0-logloss:0.33264
[485]	validation_0-logloss:0.33264
[486]	validation_0-logloss:0.33264
[487]	validation_0-logloss:0.33264
[488]	validation_0-logloss:0.33264
[489]	validation_0-logloss:0.33264
[490]	validation_0-logloss:0.33264
[491]	validation_0-logloss:0.33264
[492]	validation_0-logloss:0.33264
[493]	validation_0-logloss:0.33264
[494]	validation_0-logloss:0.33264
[495]	validation_0-logloss:0.33264
[496]	validation_0-logloss:0.33264
[497]	validation_0-logloss:0.33264
[498]	validation_0-logloss:0.33264
[499]	validation_0-l

[706]	validation_0-logloss:0.33264
[707]	validation_0-logloss:0.33264
[708]	validation_0-logloss:0.33264
[709]	validation_0-logloss:0.33264
[710]	validation_0-logloss:0.33264
[711]	validation_0-logloss:0.33264
[712]	validation_0-logloss:0.33264
[713]	validation_0-logloss:0.33264
[714]	validation_0-logloss:0.33264
[715]	validation_0-logloss:0.33264
[716]	validation_0-logloss:0.33264
[717]	validation_0-logloss:0.33264
[718]	validation_0-logloss:0.33264
[719]	validation_0-logloss:0.33264
[720]	validation_0-logloss:0.33264
[721]	validation_0-logloss:0.33264
[722]	validation_0-logloss:0.33264
[723]	validation_0-logloss:0.33264
[724]	validation_0-logloss:0.33264
[725]	validation_0-logloss:0.33264
[726]	validation_0-logloss:0.33264
[727]	validation_0-logloss:0.33264
[728]	validation_0-logloss:0.33264
[729]	validation_0-logloss:0.33264
[730]	validation_0-logloss:0.33264
[731]	validation_0-logloss:0.33264
[732]	validation_0-logloss:0.33264
[733]	validation_0-logloss:0.33264
[734]	validation_0-l

[140]	validation_0-logloss:0.20349
[141]	validation_0-logloss:0.20349
[142]	validation_0-logloss:0.20349
[143]	validation_0-logloss:0.20349
[144]	validation_0-logloss:0.20349
[145]	validation_0-logloss:0.20349
[146]	validation_0-logloss:0.20349
[147]	validation_0-logloss:0.20349
[148]	validation_0-logloss:0.20349
[149]	validation_0-logloss:0.20349
[150]	validation_0-logloss:0.20349
[151]	validation_0-logloss:0.20349
[152]	validation_0-logloss:0.20349
[153]	validation_0-logloss:0.20349
[154]	validation_0-logloss:0.20349
[155]	validation_0-logloss:0.20349
[156]	validation_0-logloss:0.20349
[157]	validation_0-logloss:0.20349
[158]	validation_0-logloss:0.20349
[159]	validation_0-logloss:0.20349
[160]	validation_0-logloss:0.20349
[161]	validation_0-logloss:0.20349
[162]	validation_0-logloss:0.20349
[163]	validation_0-logloss:0.20349
[164]	validation_0-logloss:0.20349
[165]	validation_0-logloss:0.20349
[166]	validation_0-logloss:0.20349
[167]	validation_0-logloss:0.20349
[168]	validation_0-l

...

[330]	validation_0-logloss:0.29024
[331]	validation_0-logloss:0.28984
[332]	validation_0-logloss:0.28997
[333]	validation_0-logloss:0.29032
[334]	validation_0-logloss:0.29000
[335]	validation_0-logloss:0.29009
[336]	validation_0-logloss:0.28971
[337]	validation_0-logloss:0.28927
[338]	validation_0-logloss:0.28946
[339]	validation_0-logloss:0.28902
[340]	validation_0-logloss:0.28950
[341]	validation_0-logloss:0.28889
[342]	validation_0-logloss:0.28815
[343]	validation_0-logloss:0.28803
[344]	validation_0-logloss:0.28754
[345]	validation_0-logloss:0.28740
[346]	validation_0-logloss:0.28695
[347]	validation_0-logloss:0.28670
[348]	validation_0-logloss:0.28671
[349]	validation_0-logloss:0.28652
[350]	validation_0-logloss:0.28630
[351]	validation_0-logloss:0.28581
[352]	validation_0-logloss:0.28577
[353]	validation_0-logloss:0.28593
[354]	validation_0-logloss:0.28566
[355]	validation_0-logloss:0.28525
[356]	validation_0-logloss:0.28535
[357]	validation_0-logloss:0.28495
[358]	validation_0-l

[565]	validation_0-logloss:0.25700
[566]	validation_0-logloss:0.25721
[567]	validation_0-logloss:0.25732
[568]	validation_0-logloss:0.25686
[569]	validation_0-logloss:0.25697
[570]	validation_0-logloss:0.25699
[571]	validation_0-logloss:0.25712
[572]	validation_0-logloss:0.25708
[573]	validation_0-logloss:0.25734
[574]	validation_0-logloss:0.25751
[575]	validation_0-logloss:0.25753
[576]	validation_0-logloss:0.25732
[577]	validation_0-logloss:0.25704
[578]	validation_0-logloss:0.25718
[579]	validation_0-logloss:0.25739
[580]	validation_0-logloss:0.25738
[581]	validation_0-logloss:0.25691
[582]	validation_0-logloss:0.25693
[583]	validation_0-logloss:0.25699
[584]	validation_0-logloss:0.25679
[585]	validation_0-logloss:0.25676
[586]	validation_0-logloss:0.25675
[587]	validation_0-logloss:0.25642
[588]	validation_0-logloss:0.25646
[589]	validation_0-logloss:0.25623
[590]	validation_0-logloss:0.25630
[591]	validation_0-logloss:0.25607
[592]	validation_0-logloss:0.25602
[593]	validation_0-l

[99]	validation_0-logloss:0.21369
[100]	validation_0-logloss:0.21369
[101]	validation_0-logloss:0.21369
[102]	validation_0-logloss:0.21369
[103]	validation_0-logloss:0.21369
[104]	validation_0-logloss:0.21369
[105]	validation_0-logloss:0.21369
[106]	validation_0-logloss:0.21369
[107]	validation_0-logloss:0.21369
[108]	validation_0-logloss:0.21369
[109]	validation_0-logloss:0.21369
[110]	validation_0-logloss:0.21369
[111]	validation_0-logloss:0.21369
[112]	validation_0-logloss:0.21370
[113]	validation_0-logloss:0.21370
[114]	validation_0-logloss:0.21370
[115]	validation_0-logloss:0.21370
[116]	validation_0-logloss:0.21370
[117]	validation_0-logloss:0.21225
[118]	validation_0-logloss:0.21225
[119]	validation_0-logloss:0.21226
[120]	validation_0-logloss:0.21226
[121]	validation_0-logloss:0.21227
[122]	validation_0-logloss:0.21227
[123]	validation_0-logloss:0.21228
[124]	validation_0-logloss:0.21208
[125]	validation_0-logloss:0.21208
[126]	validation_0-logloss:0.21208
[127]	validation_0-lo

[334]	validation_0-logloss:0.20348
[335]	validation_0-logloss:0.20337
[336]	validation_0-logloss:0.20337
[337]	validation_0-logloss:0.20337
[338]	validation_0-logloss:0.20337
[339]	validation_0-logloss:0.20337
[340]	validation_0-logloss:0.20336
[341]	validation_0-logloss:0.20336
[342]	validation_0-logloss:0.20336
[343]	validation_0-logloss:0.20336
[344]	validation_0-logloss:0.20336
[345]	validation_0-logloss:0.20336
[346]	validation_0-logloss:0.20336
[347]	validation_0-logloss:0.20336
[348]	validation_0-logloss:0.20336
[349]	validation_0-logloss:0.20336
[350]	validation_0-logloss:0.20336
[351]	validation_0-logloss:0.20335
[352]	validation_0-logloss:0.20335
[353]	validation_0-logloss:0.20335
[354]	validation_0-logloss:0.20335
[355]	validation_0-logloss:0.20335
[356]	validation_0-logloss:0.20335
[357]	validation_0-logloss:0.20335
[358]	validation_0-logloss:0.20335
[359]	validation_0-logloss:0.20335
[360]	validation_0-logloss:0.20335
[361]	validation_0-logloss:0.20335
[362]	validation_0-l

[569]	validation_0-logloss:0.20335
[570]	validation_0-logloss:0.20335
[571]	validation_0-logloss:0.20335
[572]	validation_0-logloss:0.20335
[573]	validation_0-logloss:0.20335
[574]	validation_0-logloss:0.20335
[575]	validation_0-logloss:0.20335
[576]	validation_0-logloss:0.20335
[577]	validation_0-logloss:0.20335
[578]	validation_0-logloss:0.20335
[579]	validation_0-logloss:0.20335
[580]	validation_0-logloss:0.20335
[581]	validation_0-logloss:0.20335
[582]	validation_0-logloss:0.20335
[583]	validation_0-logloss:0.20335
[584]	validation_0-logloss:0.20335
[585]	validation_0-logloss:0.20335
[586]	validation_0-logloss:0.20335
[587]	validation_0-logloss:0.20335
[588]	validation_0-logloss:0.20335
[589]	validation_0-logloss:0.20335
[590]	validation_0-logloss:0.20335
[591]	validation_0-logloss:0.20335
[592]	validation_0-logloss:0.20335
[593]	validation_0-logloss:0.20335
[594]	validation_0-logloss:0.20335
[595]	validation_0-logloss:0.20335
[596]	validation_0-logloss:0.20335
[597]	validation_0-l

In [7]:

# Get the best model
best_model = random_search.best_estimator_

# Evaluate the best model on the test set
y_test_pred = best_model.predict(X_test_scaled)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)


# Evaluate the best model on the validation set
y_val_pred = best_model.predict(X_val_scaled)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)


Test Accuracy: 0.9487179487179487
Validation Accuracy: 0.9230769230769231
